In [ ]:
from os import getcwd
from owlready2 import default_world, onto_path
from pandas import DataFrame, Series, to_datetime, notnull, read_csv
from matplotlib.pyplot import subplots, title, show
from numpy import *

# Initialize onotlogy

In [ ]:
CURRENT_PATH = getcwd()
DATABASE_PATH = CURRENT_PATH + "/4WD_car_db_old.sqlite3"
ONTO_IRI = "http://www.semanticweb.org/fenrir/ontologies/2023/5/4WD_car_specific_ontology#"

def get_onto():
    default_world.set_backend(filename=DATABASE_PATH, exclusive=False)
    onto_path.append(CURRENT_PATH)
    return default_world.get_ontology(ONTO_IRI).load()

def close():
    default_world.close()

onto = get_onto()

# Battery discharge (OLD)

Recover all battery measures in battery_df

In [ ]:
battery_df = DataFrame(columns=['timestamp'])
battery_df.set_index('timestamp', inplace=True)

for battery in onto.search(type = onto.battery):
    for measure in battery.hasMeasure:
        res = Series(dtype=object)
        for prop in measure.get_properties():
            for value in prop[measure]:
                if prop.python_name == 'charge' : continue
                res[prop.python_name] = value
        timestamp = res.pop('timestamp')
        battery_df.loc[timestamp,res.keys().values] = res.values

In [ ]:
print(battery_df.shape)
battery_df.reset_index(inplace=True)
battery_df.rename(columns={'index': 'timestamp'}, inplace=True)
battery_df.head()

## Visualization

Here we want to visualize all the discharges patterns in one graph.
So we use a Threshold value that will separate each cycle of discharge such as if two consecutive values are more than 10 minutes separated
they are part of two different cycles.

In [ ]:
THRESHOLD = 600

battery_df['diff_time'] = battery_df['timestamp'].diff()
battery_df['cycle'] = (battery_df['diff_time'] > THRESHOLD).cumsum()
battery_df['timestamp'] = battery_df['timestamp'] - battery_df.groupby('cycle')['timestamp'].transform('min')
discharge_cycles = battery_df.groupby('cycle')

fig, ax = subplots()

for n, cycle in discharge_cycles:
    ax.plot(cycle.timestamp, cycle.voltage)
    pass

ax.set_xlabel('Timestamp')
ax.set_ylabel('Voltage')
title('Discharge patterns')

show()
discharge_cycles.describe()

# All data extracting (OLD)

Now we want to extract every measure.
With this version we ignore charge because it is only a bad calculation with the voltage value
We ignore position because we will not use it finally we ignore samplin_rate for the same reason and also we can find it with timestamps.

In [ ]:
values_to_sum = ['speed','red_intensity','green_intensity','blue_intensity']
columns = values_to_sum.copy()
columns.insert(0,'timestamp')
df = DataFrame(columns=columns)
df.set_index('timestamp',inplace=True)

for measure in onto.search(type = onto.measure):
    res = Series(dtype=object)
    for prop in measure.get_properties():
        for value in prop[measure]:
            if prop.python_name in ['charge','position','samplin_rate']: continue
            res[prop.python_name] = value
    timestamp = res.pop('timestamp')

    for value_to_sum in values_to_sum:
        if value_to_sum in res:
            if notnull(df.at[timestamp, value_to_sum]):
                value = abs(res.pop(value_to_sum))
                df.loc[timestamp,value_to_sum] += value

    df.loc[timestamp,res.keys().values] = res.values

In [ ]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'timestamp'}, inplace=True)
df['timestamp'] = to_datetime(df['timestamp'], unit='s')
df = df.replace({True: 1, False: 0})

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.to_csv(CURRENT_PATH+'/car_data_from_old_version.csv', index=False)

In [ ]:
close()